<a href="https://colab.research.google.com/github/univerself/igames/blob/main/text/fin_project/sentiment%20c6w3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ тональности отзывов на фильмы: cоревнование по сентимент-анализу

## Введение

### Постановка задачи

В этом задании вам нужно воспользоваться опытом предыдущих недель, чтобы побить бейзлайн в соревновании по сентимент-анализу отзывов на товары на Kaggle Inclass:

https://www.kaggle.com/c/simplesentiment

В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В качестве метрики качества используется accuracy. 

### План выполнения

Мы проверим следующие модели:

1. базовый для многих задач обработки естественного языка наивный байесовский классификатор;
2. линейный классификатор на логистической регрессии, использующий предобработанный текст;
3. линейный классификатор с использованием векторных представлений слов;
4. дообученная нейросетевая модель-трансформер на основе BERT.

### Условия воспроизведения

Использованы следующие версии библиотек:

In [3]:
#Если не установлен пакет для подбора гиперпараметров Optuna, раскомментируйте подходящую строку и выполните ячейку

!pip install optuna
# !conda install -c conda-forge optuna

     |████████████████████████████████| 308 kB 7.8 MB/s 
     |████████████████████████████████| 209 kB 70.4 MB/s 
     |████████████████████████████████| 80 kB 9.7 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 149 kB 72.3 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 112 kB 76.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=a04b6f47901b0fbef870a50db9bb907528113d53fdd9dd23c975a85e1fa7d638
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
# Если не установлен пакет для работы с естественным языком spacy, можно установить через pip:
!pip install -U spacy
# или Anaconda  (может задать вопрос, так что лучше открыть терминал):
# conda install -c conda-forge spacy
# Потом поставить языковую модель английского языка.
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [20]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import spacy
import optuna

print(
    "NumPy", np.__version__,
    "Pandas", pd.__version__,
    "Sci-Kit Learn", sklearn.__version__,
    "NLTK", nltk.__version__,
    "SpaCy", spacy.__version__,
    "Optuna", optuna.__version__
)

NumPy 1.19.5 Pandas 1.1.5 Sci-Kit Learn 1.0.1 NLTK 3.2.5 SpaCy 2.2.4 Optuna 2.10.0


## Сбор и первичный анализ данных

In [4]:
# Подключение API Kaggle. Ячейку можно не выполнять, если данные уже скачаны и размещены в папке с блокнотом
# !pip install -q kaggle # Раскомментируйте, если API Kaggle не установлен
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-12 11:59:54          18549  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          11216  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           5752  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3956  
nickuzmenkov/nih-chest-xrays-tfrecords    

In [5]:
# Непосредственно загружаем набор данных. Ячейку можно не выполнять, если данные уже скачаны и размещены в папке с блокнотом
!kaggle competitions download -c simplesentiment

  0% 0.00/2.83k [00:00<?, ?B/s]
100% 2.83k/2.83k [00:00<00:00, 5.44MB/s]
  0% 0.00/50.9k [00:00<?, ?B/s]
100% 50.9k/50.9k [00:00<00:00, 53.6MB/s]
  0% 0.00/193k [00:00<?, ?B/s]
100% 193k/193k [00:00<00:00, 57.0MB/s]


In [6]:
# Загружаем данные
df_marked = pd.read_csv('products_sentiment_train.tsv', sep = '\t', header = None, names = ['text', 'y'])
print ("Количество размеченных отзывов - %d, в т.ч. позитивных  %d (%0.1f%%)" % 
       (df_marked.shape[0], df_marked.y.sum(), 100.*df_marked.y.mean()))

df_answer = pd.read_csv('products_sentiment_test.tsv', sep = '\t')
print ("Количество тестовых отзывов - %d" % (df_answer.shape[0]))

Количество размеченных отзывов - 2000, в т.ч. позитивных  1274 (63.7%)
Количество тестовых отзывов - 500


При обучении и кросс-валидации нужно будет учесть, что классы несбалансированы.

Посмотрим на несколько отзывов.

In [8]:
pd.set_option('max_colwidth', 300)
df_marked.head()

,text,y
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer associates ez firewall and antivirus and fell in love with a computer security system all over again .,1
2,the wrt54g plus the hga7t is a perfect solution if you need wireless coverage in a wider area or for a hard-walled house as was my case .,1
3,"i dont especially like how music files are unstructured ; basically they are just dumped into one folder with no organization , like you might have in windows explorer folders and subfolders .",0
4,i was using the cheapie pail ... and it worked ok until the opening device fell apart .,1


Тексты уже приведены к нижнему регистру, проводить такую предобработку самим не нужно. Ожидаемо, в отзывах много речи от первого лица и эмоционально окрашенных слов и словосочетаний ("fell in love", "perfect", "like"). 

Размеченные данные будут использоваться для решения 3 подзадач:

1. Обучение модели;
2. Выбор гиперпараметров (посредством кросс-валидации);
3. Оценка точности модели после подбора гиперпараметров.

Для решения последней подзадачи выделим тестовую выборку. Предварительно продублируем отрицательные отзывы, чтобы выровнять классы (конечно, это не лучший способ выравнивания, но самый простой).

In [7]:
from sklearn.model_selection import train_test_split
df = df_marked.append(df_marked[df_marked.y == 0].sample(500, random_state=0))
X_train, X_test, y_train, y_test = train_test_split(df.text, df.y, test_size=0.3, shuffle=True, random_state=0)
X_train.shape, X_test.shape


((1750,), (750,))

## Наивный байесовский классификатор

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline

Результат с параметрами по умолчанию:

In [18]:
nb_clf = Pipeline(
    [("vectorizer", CountVectorizer()),
    ("classifier", ComplementNB())]
)
cv_result = cross_val_score(nb_clf, X_train, y_train, cv=5)
print ("Средняя точность: %0.2f%%" % (100.*cv_result.mean()),
       "Среднеквадратичное отклонение: %0.2f%%" % (100.*cv_result.std()))

Средняя точность: 80.29% Среднеквадратичное отклонение: 3.61%


Подберём гиперпараметр $\alpha$:

In [21]:
param_distributions = {
    "classifier__alpha": optuna.distributions.UniformDistribution(0.0, 1.0)
}
optuna_search = optuna.integration.OptunaSearchCV(
    nb_clf, param_distributions, n_trials=5, cv=5, 
)
optuna_search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.

[I 2022-01-08 09:24:23,603] A new study created in memory with name: no-name-0c847e02-6860-4cd1-b76e-8c6012269953
[I 2022-01-08 09:24:23,792] Trial 0 finished with value: 0.8005714285714287 and parameters: {'classifier__alpha': 0.7467890812042345}. Best is trial 0 with value: 0.8005714285714287.
[I 2022-01-08 09:24:23,987] Trial 1 finished with value: 0.804 and parameters: {'classifier__alpha': 0.26511299739209526}. Best is trial 1 with value: 0.804.
[I 2022-01-08 09:24:24,175] Trial 2 finished with value: 0.7885714285714285 and parameters: {'classifier__alpha': 0.012949003743846998}. Best is trial 1 with value: 0.804.
[I 2022-01-08 09:24:24,376] Trial 3 finished with value: 0.7982857142857143 and parameters: {'classifier__alpha': 0.6303601289967387}. Best is trial 1 with value: 0.804.
[I 2022-01-08 09:24

OptunaSearchCV(estimator=Pipeline(steps=[('vectorizer', CountVectorizer()),
                                         ('classifier', ComplementNB())]),
               n_trials=5,
               param_distributions={'classifier__alpha': UniformDistribution(high=1.0, low=0.0)})

Обучим модель на всей обучающей выборке с лучшим значением гиперпараметра и оценим точность на тестовой выборке.

In [22]:
from sklearn.metrics import classification_report
nb_clf = Pipeline(
    [("vectorizer", CountVectorizer()),
    ("classifier", ComplementNB(alpha = optuna_search.best_params_['classifier__alpha']))]
)
nb_clf.fit(X_train, y_train)
print(classification_report(y_test, nb_clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85       387
           1       0.83      0.84      0.84       363

    accuracy                           0.84       750
   macro avg       0.84      0.84      0.84       750
weighted avg       0.84      0.84      0.84       750



Получили 84% правильных ответов. Точность и полнота имеют примерно ту же величину, значит применяемого метода балансировки достаточно.

## Линейный классификатор с предобработкой текста

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

Получим сначала результат классификации на сыром тексте, без предобработки, с параметрами по умолчанию.

In [23]:
lr_clf = Pipeline(
    [("vectorizer", TfidfVectorizer()),
    ("classifier", LogisticRegression(solver='liblinear'))]
)
cv_result = cross_val_score(lr_clf, X_train, y_train, cv=5)
print ("Средняя точность: %0.2f%%" % (100.*cv_result.mean()),
       "Среднеквадратичное отклонение: %0.2f%%" % (100.*cv_result.std()))

Средняя точность: 79.54% Среднеквадратичное отклонение: 3.78%


Результат похуже, чем у наивного байесовского классификатора. Попробуем его улучшить.

### Подготовка к предобработке текста

Хотелось бы решить как минимум две задачи:

1. убрать ошибки токенизации;
2. сделать нормализацию, т.е. привести слова к форме, в которой слова "bad", "worse" и "worst" или "good", "better" и "best" для модели были одним признаком.
3. учесть словосочетания: n-граммы, а лучше и n-k-skip-граммы.

Для нормализации можно воспользоваться двумя методами:

1. стемминг - "стрижка" окончаний (применим PorterStemmer из NLTK);
2. лемматизация - поиск основы с использованием грамматических правил и словарей (для этого нам больше подойдёт инструментарий Spacy).

In [16]:
from nltk.stem.snowball import PorterStemmer

# Создадим вспомогательный анализатор на основе стеммера Портера
def porter_stemmer_analyzer(text):
    stemmer = PorterStemmer()
    analyzer = CountVectorizer().build_analyzer()
    return (stemmer.stem(word) for word in analyzer(text))

In [9]:
nlp = spacy.load("en_core_web_sm")

# Документация spaCy говорит, что коллекцию текстов лучше обрабатывать не по одному, а через метод pipe.
# Чтоб не отходить от привычного стиля организации пайплайна, обернём такую обработку в собственный трансформер.
import string
from sklearn.base import BaseEstimator, TransformerMixin
punctuations = string.punctuation

from nltk.util import skipgrams

class SpacyTransformer(TransformerMixin, BaseEstimator):
    """
    Трансформер на основе Spacy
    Выполняет токенизацию, лемматизацию текста,  построение n-грамм и n-k-skip-грамм.
    Параметры:
    check_stop_words - исключать ли стоп-слова (spaCy их определяет во время разбора текста);
    save_prons - сохранять ли исходную форму для личных местомений: в сомнительных случаях нормальной формой spaCy считает служебное слово "-PRON-";
    ngram_len - максимальная длина n-грамм;
    skipgram_k - максимальная длина пропуска n-k-skip-грамм.
    """
    def __init__(self, check_stop_words=False, save_prons=False, ngram_len=1, skipgram_k=1):
        self.save_prons = save_prons
        self.check_stop_words = check_stop_words
        self.ngram_len = ngram_len
        self.skipgram_k = skipgram_k

    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X):
        return [list(self.doc_tokens(doc)) for doc in nlp.pipe(X)]
    
    def doc_tokens(self, doc):
        """Генератор токенов с учётом n-k-skip-грамм."""
        unigrams = list(self.unigrams(doc))
        for token in unigrams:
            # Униграммы возвращаем как есть.
            yield token

        ngram_len = min(len(unigrams), self.ngram_len)
        # Более длинные словосочетания формируем в цикле.
        for n in range(2, ngram_len + 1):
            for token in skipgrams(unigrams, n, self.skipgram_k):
                yield " ".join(token)

    def unigrams(self, doc):
        """
        Генератор токенов документа Spacy. Позволяет:
        1. Отфильтровать токены,
        2. Преобразовать их к нужной форме,
        """
        for token in (token for token in doc if self.use_token(token)):
            if self.save_prons and token.lemma_ == "-PRON-":
                yield token.lower_
            yield token.lemma_.lower().strip()
    
    def use_token(self, token):
        """Метод, определяющий, какие из токенов будут использованы."""
        return not (
            self.check_stop_words and token.is_stop
        ) and token.text not in punctuations

# Чтобы векторизаторы SKlearn не делали лишнего, определим ничего не делающий токенизатор
def do_nothing(something):
    return something

In [18]:
# Проверка
pipeline_with_preprocessing = Pipeline([
    ("preprocessor", SpacyTransformer(ngram_len = 3, skipgram_k = 1)),
    ("vectorizer", CountVectorizer(tokenizer = do_nothing, preprocessor=do_nothing))
])
pipeline_with_preprocessing.fit_transform(X_train).shape

(1750, 91362)

### Подбор гиперпараметров с учётом предобработки

Гиперпараметрами в этом разделе будут:

* способ предобработки (стемминг или лемматизация);
* максимальная длина словосочетаний;
* способ векторизации (простой "мешок слов" и TF-IDF);
* пороги вхождения слов по частоте их употребления в корпусе;
* параметры регуляризации логистической регрессии.

In [19]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import warnings

def svm_objective(trial):
  steps = []

  ngram_len = trial.suggest_int("ngram_len", 1, 8)
  check_stop_words = trial.suggest_categorical("check_stop_words", [False, True])

  # Параметры препроцессинга
  normalization = trial.suggest_categorical("normalization", ["stemming", "lemmatization"])
  if(normalization == "stemming"):
    stop_words = stopwords if check_stop_words else None
  else:
    save_prons = trial.suggest_categorical("save_prons", [False, True])
    skipgram_k = trial.suggest_int("skipgram_k", 0, 8)
    steps.append(("preprocessor", SpacyTransformer(
        save_prons=save_prons, 
        check_stop_words=check_stop_words,
        ngram_len=ngram_len, 
        skipgram_k=skipgram_k)))

  # Параметры векторизации
  vectorizer_name = trial.suggest_categorical("vectorizer_name", ["CountVectorizer", "TfidfVectorizer"])
  max_df = trial.suggest_float("max_df", 0.7, 1.)
  min_df = trial.suggest_float("min_df", 0., 0.3)

  if(normalization == "stemming" and vectorizer_name == "CountVectorizer"):
    steps.append(("vectorizer", CountVectorizer(
        analyzer=porter_stemmer_analyzer,
        min_df=min_df, max_df=max_df, stop_words = stop_words,
        ngram_range=(1, ngram_len))))
  elif(normalization == "stemming"):
    steps.append(("vectorizer", TfidfVectorizer(
        analyzer=porter_stemmer_analyzer,
        min_df=min_df, max_df=max_df, stop_words = stop_words,
        ngram_range=(1, ngram_len))))
  elif(normalization == "CountVectorizer"):
    steps.append(("vectorizer", CountVectorizer(
        tokenizer = do_nothing, preprocessor=do_nothing,
        min_df=min_df, max_df=max_df)))
  else:
    steps.append(("vectorizer", TfidfVectorizer(
        tokenizer = do_nothing, preprocessor=do_nothing,
        min_df=min_df, max_df=max_df)))

  # Параметры классификатора
  svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
  steps.append(("classifier", LogisticRegression(C=svc_c, solver='liblinear')))

  clf = Pipeline(steps)

  # Некоторые комбинации параметров могут быть бессмысленными, отключим вывод предупреждений
  warnings.filterwarnings("ignore")
  cv_result = cross_val_score(clf, X_train, y_train, cv=5)
  warnings.filterwarnings("default")

  return cv_result.mean()

In [21]:
from optuna.visualization import plot_optimization_history

study = optuna.create_study(direction="maximize")

# Проверим сначала очевидные комбинации
study.enqueue_trial({
    "normalization": "lemmatization",
    "save_prons": False,
    "check_stop_words": False,
    "vectorizer_name": "TfidfVectorizer",
    'max_df': 1.0, 
    'min_df': 0.0,
    'ngram_len': 3,
    'skipgram_k': 1,
    'svc_c': 1.0
})
study.enqueue_trial({
    "normalization": "lemmatization",
    "save_prons": True,
    "check_stop_words": False,
    "vectorizer_name": "TfidfVectorizer",
    'max_df': 1.0, 
    'min_df': 0.0,
    'ngram_len': 3,
    'skipgram_k': 1,
    'svc_c': 1.0
})
study.enqueue_trial({
    "normalization": "lemmatization",
    "save_prons": False,
    "check_stop_words": True,
    "vectorizer_name": "TfidfVectorizer",
    'max_df': 1.0, 
    'min_df': 0.0,
    'ngram_len': 3,
    'skipgram_k': 1,
    'svc_c': 1.0
})
study.enqueue_trial({
    "normalization": "lemmatization",
    "save_prons": True,
    "check_stop_words": True,
    "vectorizer_name": "TfidfVectorizer",
    'max_df': 1.0, 
    'min_df': 0.0,
    'ngram_len': 3,
    'skipgram_k': 1,
    'svc_c': 1.0
})
study.enqueue_trial({
    "normalization": "stemming",
    "vectorizer_name": "TfidfVectorizer",
    'max_df': 1.0, 
    'min_df': 0.0,
    'ngram_len': 3,
    'svc_c': 1.0
})

# А теперь запустим оптимизацию с учётом указанных выше точек
study.optimize(svm_objective, n_trials=100)

print("Лучшие параметры: ", study.best_params)
plot_optimization_history(study)

[I 2022-01-08 06:23:02,534] A new study created in memory with name: no-name-ff54e71a-cca7-4bf5-a91a-a91842c6bbde
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: ExperimentalWarning:

enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:857: ExperimentalWarning:

create_trial is experimental (supported from v2.0.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:857: ExperimentalWarning:

add_trial is experimental (supported from v2.0.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: ExperimentalWarning:

enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: ExperimentalWarning:

enqueue_trial is experimental (supported from v1.2.0). The interface c

Лучшие параметры:  {'ngram_len': 3, 'check_stop_words': False, 'normalization': 'lemmatization', 'save_prons': False, 'skipgram_k': 1, 'vectorizer_name': 'TfidfVectorizer', 'max_df': 1.0, 'min_df': 0.0, 'svc_c': 1.0}


In [24]:
# Лучший вариант с лемматизацией
lr_clf = Pipeline([
    ("preprocessor", SpacyTransformer(
        save_prons=False, 
        check_stop_words=False,
        ngram_len=3, 
        skipgram_k=1)),
    ("vectorizer", TfidfVectorizer(
        tokenizer = do_nothing, preprocessor=do_nothing,
        min_df=0.0, max_df=1.0)),
    ("classifier", LogisticRegression(C=1.0, solver='liblinear'))
])
lr_clf.fit(X_train, y_train)
print(classification_report(y_test, lr_clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87       387
           1       0.84      0.91      0.87       363

    accuracy                           0.87       750
   macro avg       0.87      0.87      0.87       750
weighted avg       0.87      0.87      0.87       750



In [26]:
# Запишем текущий результат
df_answer['y'] = lr_clf.predict(df_answer.text)
df_answer[['Id','y']].to_csv('product-reviews-sentiment-analysis-lr.csv', index = False)


Результат обсуждаемой в разделе модели - 87% правильных ответов, что превосходит наивный байесовский классификатор. Однако на проверочных данных Kaggle результат меньше 80%, будем пробовать дальше.

## Логистическая регрессия с использованием векторных представлений слов;
## Дообученная нейросетевая модель-трансформер на основе BERT.

In [ ]:
# Скачиваем словари
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Простейшая модель

Хотя ожидалось, что лучший результат будет показан моделью с сильной регуляризацией и использованием триграмм, это не оправдалось.

На этом остановимся. К сожалению, качественного скачка результата достичь не удалось, хотя +3.5% точности тоже не худший результат. Эксперименты с другими простыми моделями (линейными и байесовскими), которые не вошли в итоговый ноутбук, пока также не принесли успеха. На будущее стоит попробовать учесть векторные представления слов и взаимосвязи слов.